Load jupyter sql extension


In [93]:
#Load Jupter SQL extension
%load_ext sql
%config SqlMagic.autocommit = False

In [94]:
import pandas as pd

In [95]:
#Load database
%sql 

'Connected: @../data/raw/frantamarit.db'

In [130]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../data/raw/frantamarit.db')

In [96]:
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///../data/raw/frantamarit.db
Done.
Returning data to local variable tables


In [139]:
sql = \
"SELECT rank_info.id, rank_info.name as fullName, rank_info.position,personal_info.lastName,\
personal_info.age, personal_info.gender, personal_info.country, personal_info.image,\
business_info.Source as source, business_info.worth, business_info.worthChange,\
business_info.realTimePosition \
FROM rank_info \
LEFT JOIN personal_info \
ON rank_info.id = personal_info.id \
LEFT JOIN business_info \
ON rank_info.id = business_info.id \
ORDER BY rank_info.position DESC"

df_master_new = pd.read_sql_query(sql, engine)

In [140]:
df_master_new

,id,fullName,position,lastName,age,gender,country,image,source,worth,worthChange,realTimePosition
0,1690,OLGUN Zorlu,2208.0,zoRLU,54 years old,Male,Turkey,https://specials-images.forbesimg.com/imageser...,Diversified ==> diversified,1.0 BUSD,nan millions USD,1978.0
1,2599,zhuO Jun,2207.0,zhUO,52 years old,F,Hong Kong,https://specials-images.forbesimg.com/imageser...,Manufacturing ==> printed circuit boards,1.0 BUSD,0.0 millions USD,1978.0
2,8536,ZHU XINGMING,2206.0,zHU,51 years old,None,People's Republic of China,https://specials-images.forbesimg.com/imageser...,Manufacturing ==> electrical equipment,1.0 BUSD,0.0 millions USD,1856.0
3,5887,ZHOU LIANGZHANG,2205.0,zhOU,55 years old,None,None,https://specials-images.forbesimg.com/imageser...,Manufacturing ==> electrical equipment,1.0 BUSD,nan millions USD,2134.0
4,9216,zhao XIAOQIANg,2204.0,zHAO,51 years old,M,China,https://specials-images.forbesimg.com/imageser...,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,0.0 millions USD,2134.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2255,8911,supaluck umPUJH & Family,NaN,None,None,None,None,None,None,None,None,NaN
2256,4098,thomaS Duff,NaN,None,None,None,None,None,None,None,None,NaN
2257,6885,zakhar SMUSHkin,NaN,None,None,None,None,None,None,None,None,NaN
2258,5401,zhang ZHONGNENG & family,NaN,None,None,None,None,None,None,None,None,NaN


In [97]:
%%sql master_join <<

SELECT
    rank_info.id,
    rank_info.name as fullName,
    rank_info.position,
    personal_info.lastName,
    personal_info.age,
    personal_info.gender,
    personal_info.country,
    personal_info.image,
    business_info.Source as source,
    business_info.worth,
    business_info.worthChange,
    business_info.realTimePosition
FROM 
    rank_info
LEFT JOIN 
    personal_info
ON
    rank_info.id = personal_info.id
LEFT JOIN
    business_info
ON 
    rank_info.id = business_info.id
ORDER BY
    rank_info.position DESC;

 * sqlite:///../data/raw/frantamarit.db
Done.
Returning data to local variable master_join


In [98]:
#Showing tables
df_master = master_join.DataFrame()

In [127]:
df_master

,id,fullName,position,lastName,age,gender,country,image,source,worth,worthChange,realTimePosition,sourceDetails
0,1690,olgun zorlu,2208.0,zorlu,54.0,2,turkey,https://specials-images.forbesimg.com/imageser...,diversified,1.0,0.0,1978.0,diversified
1,2599,zhuo jun,2207.0,zhuo,52.0,1,hong kong,https://specials-images.forbesimg.com/imageser...,manufacturing,1.0,0.0,1978.0,printed circuit boards
2,8536,zhu xingming,2206.0,zhu,51.0,0,china,https://specials-images.forbesimg.com/imageser...,manufacturing,1.0,0.0,1856.0,electrical equipment
3,5887,zhou liangzhang,2205.0,zhou,55.0,0,none,https://specials-images.forbesimg.com/imageser...,manufacturing,1.0,0.0,2134.0,electrical equipment
4,9216,zhao xiaoqiang,2204.0,zhao,51.0,2,china,https://specials-images.forbesimg.com/imageser...,fashion & retail,1.0,0.0,2134.0,"fashion, entertainment"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255,8911,supaluck umpujh & family,0.0,,0.0,0,,,,0.0,0.0,0.0,
2256,4098,thomas duff,0.0,,0.0,0,,,,0.0,0.0,0.0,
2257,6885,zakhar smushkin,0.0,,0.0,0,,,,0.0,0.0,0.0,
2258,5401,zhang zhongneng & family,0.0,,0.0,0,,,,0.0,0.0,0.0,


In [116]:
#To use afterwards
def lowercase_feature_1(df,col):
    return df[col].str.lower()

In [117]:
to_lower_cols = ['lastName', 'fullName', 'country', 'source']

for i in to_lower_cols:
    df_master[i] = lowercase_feature_1(df_master, i)

In [100]:
def extract_str(df, col, regex):
    return df[col].str.extract(regex)

def extract_float(df, col, regex):
    return df[col].str.extract(regex).astype('float64')

In [101]:
df_master['sourceDetails'] = extract_str(df_master,'source',r"> (.*)")

In [102]:
df_master['source'] = extract_str(df_master,'source',r"(.*)  =")

In [103]:
#In USD Billion
df_master['worth'] = extract_float(df_master,'worth',r"(\d+.?\d)")

In [104]:
#In USD Billion
df_master['worthChange'] = extract_float(df_master,'worthChange',r"(-?\d+.?\d*) ")

In [105]:
df_master['country'].replace("People's Republic of China", 'China',inplace=True)

In [106]:
#Using Float but better to be treated as integer later on
df_master['age'] = extract_float(df_master,'age',r'(\d+)')

In [107]:
df_master['age'] = [2018 - i if i > 1000 else i for i in df_master['age']]

In [109]:
#1 assigned to Female, 2 to Male & 0 to nulls.
df_master['gender'] = [1 if i == 'F' or i == 'Female' else 2 if i == 'M' or i == 'Male' else 0 for i in df_master['gender']]

In [115]:
df_master['fullName'].sort_values().unique()

array(['ABHAY FIRodia', 'ABIGAIL JOHNson', 'AGNETE KIRK THINGGAARd', ...,
       'zhuanG KUilong', 'zonG QINGHOU', 'zuO Hui'], dtype=object)

In [124]:
for i in df_master.columns:
    print(i)

id
fullName
position
lastName
age
gender
country
image
source
worth
worthChange
realTimePosition
sourceDetails


In [125]:
for i,j in zip(df_master.dtypes, df_master.columns):
    if i == 'float64' or i == 'int64':
        df_master[j].fillna(0, inplace=True)
    else:
        df_master[j].fillna('', inplace=True)

In [128]:
df_master.to_csv(f'../data/processed/cleaned_output.csv', sep=';', index=False)

In [20]:
#We don't want to get rid of the NaN yet, so we'll fill them with 0s.
def fill_0(x):
    return x.fillna(0, inplace=True)

fill_0(df_rank_info)
df_rank_info

,id,name,position
0,5542,anant asavabhokhin & family,0.0
1,4183,arash ferdowsi,0.0
2,4761,banthoon lamsam & family,0.0
3,1449,bill adderley & family,0.0
4,1624,boris zingarevich,0.0
...,...,...,...
2255,9216,zhao xiaoqiang,2204.0
2256,5887,zhou liangzhang,2205.0
2257,8536,zhu xingming,2206.0
2258,2599,zhuo jun,2207.0


In [112]:
#get nulls

def get_null(x):
    return x.isnull().sum()

In [126]:
get_null(df_master)

id                  0
fullName            0
position            0
lastName            0
age                 0
gender              0
country             0
image               0
source              0
worth               0
worthChange         0
realTimePosition    0
sourceDetails       0
dtype: int64

In [119]:
for i in df_master.dtypes:
    if i == 'float64':
        print('yest')
    else:
        print('no')

no
no
yest
no
yest
no
no
no
no
yest
yest
yest
no


In [120]:
df_master.dtypes

id                    int64
fullName             object
position            float64
lastName             object
age                 float64
gender                int64
country              object
image                object
source               object
worth               float64
worthChange         float64
realTimePosition    float64
sourceDetails        object
dtype: object

In [ ]:
personal_info.head()

In [ ]:
master_info.isnull().sum()